In [3]:
import requests
import psycopg2
from datetime import datetime
from dotenv import load_dotenv
import os
import pandas as pd


# Carga las variables de entorno desde .env
load_dotenv()

# Construye la cadena de conexión usando las variables de entorno
REDSHIFT_ENDPOINT = os.getenv('REDSHIFT_ENDPOINT')
REDSHIFT_DB = os.getenv('REDSHIFT_DB')
REDSHIFT_USER = os.getenv('REDSHIFT_USER')
REDSHIFT_PASSWORD = os.getenv('REDSHIFT_PASSWORD')
REDSHIFT_PORT = os.getenv('REDSHIFT_PORT')

# Construye la cadena de conexión
conn_string = f"dbname='{REDSHIFT_DB}' port='{REDSHIFT_PORT}' user='{REDSHIFT_USER}' password='{REDSHIFT_PASSWORD}' host='{REDSHIFT_ENDPOINT}'"

try:
    # Intenta establecer la conexión
    conn = psycopg2.connect(conn_string)
    print("Conexión establecida correctamente.")
    # Crear la tabla si no existe
    cursor = conn.cursor()
    cursor.execute("""
             CREATE TABLE IF NOT EXISTS crypto_markets (
                 id VARCHAR(50),
                 symbol VARCHAR(10),
                 name VARCHAR(50),
                 image_url VARCHAR(255),
                 current_price DECIMAL(18,2),
                 market_cap BIGINT,
                 market_cap_rank INT,
                 total_volume BIGINT,
                 high_24h DECIMAL(18,2),
                 low_24h DECIMAL(18,2),
                 price_change_24h DECIMAL(18,2),
                 price_change_percentage_24h DECIMAL(18,2),
                 last_updated TIMESTAMP
             );
         """)
    conn.commit()
    print("Tabla Creada correctamente.")

except psycopg2.Error as e:
    # Manejo de excepciones para capturar errores en la conexión
    print("Error al conectar a Redshift o crear la tabla:", e)


Conexión establecida correctamente.
Tabla Creada correctamente.


In [4]:
try:
    # Vaciar la tabla para evitar duplicados o inconsistencias
    cursor.execute("Truncate table crypto_markets")
    count_entries = cursor.rowcount
    print(count_entries)
except psycopg2.Error as e:
    # Manejo de excepciones para capturar errores en la conexión
    print("Error al truncar la tabla:", e)
    cursor.close()
    conn.close()

-1


In [14]:
# Consultando la tabla previo al insert
try:
    cursor.execute("SELECT * FROM crypto_markets")
    results = cursor.fetchall()
    print(results)
except psycopg2.Error as e:
    # Manejo de excepciones para capturar errores en la conexión
    print("Error al obtener datos de la tabla:", e)
    cursor.close()
    conn.close()

[('bitcoin', 'btc', 'Bitcoin', 'https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', Decimal('62798.00'), 1231834718463, 1, 23869289687, Decimal('63234.00'), Decimal('61483.00'), Decimal('862.25'), Decimal('1.39'), datetime.datetime(2024, 3, 3, 23, 10, 0, 95000)), ('ethereum', 'eth', 'Ethereum', 'https://assets.coingecko.com/coins/images/279/large/ethereum.png?1696501628', Decimal('3470.98'), 415995061103, 2, 14822483579, Decimal('3494.38'), Decimal('3376.02'), Decimal('54.04'), Decimal('1.58'), datetime.datetime(2024, 3, 3, 23, 9, 59, 806000)), ('tether', 'usdt', 'Tether', 'https://assets.coingecko.com/coins/images/325/large/Tether.png?1696501661', Decimal('1.00'), 99296895149, 3, 59346501462, Decimal('1.00'), Decimal('1.00'), Decimal('0.00'), Decimal('-0.17'), datetime.datetime(2024, 3, 3, 23, 5, 1, 854000)), ('binancecoin', 'bnb', 'BNB', 'https://assets.coingecko.com/coins/images/825/large/bnb-icon2_2x.png?1696501970', Decimal('414.41'), 63645339893, 4, 1307978

In [13]:
try:
    # Llenar la Tabla
    # URL de la API de CoinGecko
    api_url = 'https://api.coingecko.com/api/v3/coins/markets'
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 10,  # Limitar el número de resultados
        'page': 1,
        'sparkline': 'false'
    }

    # Realizar la solicitud a la API
    response = requests.get(api_url, params=params)

    # Verificar que la respuesta es exitosa
    if response.status_code == 200:
        data = response.json()

        # Insertar datos en la base de datos
        for item in data:
            cursor.execute("""
                INSERT INTO crypto_markets (
                    id, symbol, name, image_url, current_price, market_cap, market_cap_rank,
                    total_volume, high_24h, low_24h, price_change_24h, price_change_percentage_24h, last_updated
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                item['id'], item['symbol'], item['name'], item['image'], item['current_price'],
                item['market_cap'], item['market_cap_rank'], item['total_volume'],
                item['high_24h'], item['low_24h'], item['price_change_24h'], item['price_change_percentage_24h'],
                datetime.fromisoformat(item['last_updated'].replace('Z', '+00:00'))  # Asegurar el formato correcto de timestamp
            ))
        conn.commit()
        print("Datos insertados correctamente")
    else:
        print("Failed to retrieve data", response.status_code)

except psycopg2.Error as e:
    # Manejo de excepciones para capturar errores en la conexión
    print("Error al insertar datos:", e)

# Cerrar la conexión a la base de datos
cursor.close()
conn.close()

In [24]:
# Configuración de conexión a la base de datos
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
# Realizar la consulta
query = "SELECT * FROM crypto_markets"

# Crear un DataFrame vacío en caso de error
df = pd.DataFrame()

# Ejecutar la consulta
cursor.execute(query)
    
# Recuperar los resultados
results = cursor.fetchall()
    
# Convertir los resultados en un DataFrame de Pandas
# Los nombres de las columnas se pueden recuperar con cursor.description
colnames = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=colnames)
    
# Muestra las primeras filas del DataFrame
df.head()

,id,symbol,name,image_url,current_price,market_cap,market_cap_rank,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,62798.00,1231834718463,1,23869289687,63234.00,61483.00,862.25,1.39,2024-03-03 23:10:00.095
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,3470.98,415995061103,2,14822483579,3494.38,3376.02,54.04,1.58,2024-03-03 23:09:59.806
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.00,99296895149,3,59346501462,1.00,1.00,0.00,-0.17,2024-03-03 23:05:01.854
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,414.41,63645339893,4,1307978149,417.29,405.50,4.36,1.06,2024-03-03 23:09:51.628
4,solana,sol,Solana,https://assets.coingecko.com/coins/images/4128...,130.60,57822017171,5,3334746854,131.70,125.88,1.50,1.16,2024-03-03 23:09:51.006


In [25]:
# Cerrar la conexión a la base de datos
cursor.close()
conn.close()